# 評価方法

分類されていないデータを認識し、どれだけ正しくカテゴリごとに分類できるかを算出した「平均精度」の高さを競い合います。

今回、活用するデータはLSWMD_25519となります。
LSWMD_25519のFailureType項目が分類されていない状態のデータに対し、正しいFailureTypeカテゴリを分類するプログラムを作成し、その平均精度を算出します。
平均精度とは、カテゴリごとに正しく分類できる精度を平均した値です。カテゴリごとに算出した精度（Aが正しく分類された数/Aのデータ数）を足し、カテゴリ数で割ります。

公平な評価を実施するために、以下の制限を設けています。
1. 外部パッケージをインストールするためのセルとsolution関数の中身のみを編集すること
2. 校舎のiMac上で最後のセルの実行時間が15分未満であること　（%%timeitの出力結果を確認してください）

※気になる点がある場合、Discordで気軽にお問合せください。

In [1]:
import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from sklearn.model_selection import train_test_split

外部パッケージを使用する場合、以下の方法でインストールを実施してください。

In [2]:
# 必要な外部パッケージは、以下の内容を編集しインストールしてください
# !pip install keras
# !pip install tensorflow[and-cuda]
!python3.10 -m pip install --upgrade pip
!pip install opencv-python
!pip install scikit-image

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
import cv2
import re
import numpy as np
import pandas as pd
import pickle
from skimage.feature.texture import graycomatrix, graycoprops
from sklearn.ensemble import RandomForestClassifier

# 1, 基本統計量を計算
def calculate_statistics(array):
    array = np.array(array)
    stats = {
        'stat_mean': np.mean(array),
        'stat_std': np.std(array),
        'stat_median': np.median(array),
        'stat_min': np.min(array),
        'stat_max': np.max(array)
    }
    return stats

# 2, 画像テクスチャ特徴量を計算
def calculate_texture_features(array):
    glcm = graycomatrix(array, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    texture_features = {
        'texture_feature_contrast': graycoprops(glcm, 'contrast')[0, 0],
        'texture_feature_correlation': graycoprops(glcm, 'correlation')[0, 0],
        'texture_feature_energy': graycoprops(glcm, 'energy')[0, 0],
        'texture_feature_homogeneity': graycoprops(glcm, 'homogeneity')[0, 0]
    }
    return texture_features

# 3, 形状特徴量を計算
def scale_to_grayscale(array):
    scaled_array = np.clip(array * 127.5, 0, 255).astype(np.uint8)
    return scaled_array

def calculate_shape_features(array):
    grayscale = scale_to_grayscale(array)
    edges = cv2.Canny(grayscale, 100, 200)
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    shape_features = {'shape_feature_area': 0, 'shape_feature_perimeter': 0, 'shape_feature_complexity': 0}
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        complexity = perimeter ** 2 / (4 * np.pi * area) if area > 0 else 0
        shape_features['shape_feature_area'] += area
        shape_features['shape_feature_perimeter'] += perimeter
        shape_features['shape_feature_complexity'] += complexity
    return shape_features

# 4, フーリエ変換の計算
def calculate_fourier_transform_features(array):
    f_shift = np.fft.fftshift(np.fft.fft2(array))
    magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1e-10)
    features = {
        'feature_mean': np.mean(magnitude_spectrum),
        'feature_std': np.std(magnitude_spectrum),
        'feature_median': np.median(magnitude_spectrum),
        'feature_min': np.min(magnitude_spectrum),
        'feature_max': np.max(magnitude_spectrum)
    }
    return features

# 特徴量を適用する関数
def apply_features(df):
    df['waferMap_statistics'] = df['waferMap'].apply(calculate_statistics)
    df['waferMap_texture_features'] = df['waferMap'].apply(lambda x: calculate_texture_features(np.array(x)))
    df['waferMap_shape_features'] = df['waferMap'].apply(calculate_shape_features)
    df['waferMap_fourier_features'] = df['waferMap'].apply(calculate_fourier_transform_features)
    return df

# 辞書形式の特徴量を展開する関数
def expand_features(df, feature_column):
    features_df = pd.DataFrame(df[feature_column].tolist(), index=df.index)
    df = pd.concat([df.drop(columns=[feature_column]), features_df], axis=1)
    return df

# lotName から数値部分を抽出して整数に変換する関数
def extract_number(lot_name):
    numbers = re.findall(r'\d+', lot_name)
    return int(numbers[0]) if numbers else None

# solution関数の更新
# solution関数の更新
def solution(x_test_df, train_df):
    # 特徴量を適用
    train_df = apply_features(train_df)
    x_test_df = apply_features(x_test_df)

    # 特徴量を展開
    for feature in ['waferMap_statistics', 'waferMap_texture_features', 'waferMap_shape_features', 'waferMap_fourier_features']:
        train_df = expand_features(train_df, feature)
        x_test_df = expand_features(x_test_df, feature)

    # lotName から数値部分を抽出して整数に変換
    train_df['lotNumber'] = train_df['lotName'].apply(extract_number)
    x_test_df['lotNumber'] = x_test_df['lotName'].apply(extract_number)

    # 特徴量とターゲットの分離
    X_train = train_df.drop(['failureType', 'waferMap', 'lotName'], axis=1)
    y_train = train_df['failureType']

    # モデルの訓練
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # テストデータでの予測
    X_test = x_test_df.drop(['waferMap', 'lotName'], axis=1)
    predictions = model.predict(X_test)

    return pd.DataFrame({'failureType': predictions}, index=x_test_df.index)




solution関数は以下のように活用され、平均精度を計算します。

In [4]:
%%timeit -r 1 -n 1

# データのインポート
df=pd.read_pickle("../input/LSWMD_25519.pkl")

# テスト用と学習用のデータを作成（テストする際は、random_stateの値などを編集してみてください）
train_df, test_df = train_test_split(df, stratify=df['failureType'], test_size=0.10, random_state=42)

y_test_df = test_df[['failureType']]
x_test_df = test_df.drop(columns=['failureType'])

# solution関数を実行
user_result_df = solution(x_test_df, train_df)

average_accuracy = 0
# ユーザーの提出物のフォーマット確認
if type(y_test_df) == type(user_result_df) and y_test_df.shape == user_result_df.shape:
    # 平均精度の計算
    accuracies = {}
    for failure_type in df['failureType'].unique():
        y_test_df_by_failure_type = y_test_df[y_test_df['failureType'] == failure_type]
        user_result_df_by_failure_type = user_result_df[y_test_df['failureType'] == failure_type]
        matching_rows = (y_test_df_by_failure_type == user_result_df_by_failure_type).all(axis=1).sum()
        accuracies[failure_type] = (matching_rows/(len(y_test_df_by_failure_type)))
    
    average_accuracy = sum(accuracies.values())/len(accuracies)

print(f"平均精度：{average_accuracy*100:.2f}%")

平均精度：73.47%
1min 31s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
